In [4]:
import sys
print(sys.path)
sys.path.append('/lib/python3.7/site-packages')
from PIL import Image

['C:\\Users\\theop\\OneDrive\\Documents\\GitHub\\digit_fashion_nn\\dl_2020', 'C:\\Users\\theop\\Anaconda3\\python37.zip', 'C:\\Users\\theop\\Anaconda3\\DLLs', 'C:\\Users\\theop\\Anaconda3\\lib', 'C:\\Users\\theop\\Anaconda3', '', 'C:\\Users\\theop\\AppData\\Roaming\\Python\\Python37\\site-packages', 'C:\\Users\\theop\\Anaconda3\\lib\\site-packages', 'c:\\users\\theop\\onedrive\\documents\\github\\ada_project\\src', 'C:\\Users\\theop\\Anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\theop\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\theop\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\theop\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\theop\\.ipython']


In [3]:
import torch
import math
from torch.nn import functional as F
import dlc_practical_prologue as prologue

import matplotlib.pyplot as plt
%matplotlib inline

from torch import optim
from torch import Tensor
from torch import nn
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(device)
else:
    device = torch.device('cpu')
    print(device)


ImportError: cannot import name 'PILLOW_VERSION' from 'PIL' (C:\Users\theop\Anaconda3\lib\site-packages\PIL\__init__.py)

In [2]:
#Generating pairs of 14x14
N=1000
(train_input,train_target,train_classes, \
 test_input,test_target,test_classes) = prologue.generate_pair_sets(N)

train_input = train_input.to(device)
test_input = test_input.to(device)
train_target = train_target.to(device)
test_target = test_target.to(device)
print(train_input.size())
print(train_target.size())
print(train_classes.size())
print(test_input.size())
print(test_target.size())
print(test_classes.size())
print(test_input.narrow(0,200,100).size())

NameError: name 'prologue' is not defined

In [5]:
def norm_(train_input,test_input):
    """Function to normalize the input"""
    mu, std = train_input.mean(), train_input.std()
    train_inputOut = train_input.sub_(mu).div_(std)
    test_inputOut = test_input.sub_(mu).div_(std)
    return train_inputOut, test_inputOut

In [6]:
train_input,test_input = norm_(train_input,test_input);
print(train_input.size())
print(train_input.device)


torch.Size([1000, 2, 14, 14])
cuda:0


### Models :
ConvNet (LeNet)

In [ ]:
#defining model
class testNet(nn.Module):
    def __init__(self,nb_hidden):
        super(testNet, self).__init__()
        #takes 2x14x14
        #gives 32x12x12, 32x6x6 after maxpool
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3) 
        #takes 32x6x6, gives 64x4x4 then 64x2x2 after maxpool
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        #takes 64*2*2 = 254, gives nb_hidden
        self.fc1 = nn.Linear(2*2*64, nb_hidden)
        #takes nb_hidden, gives 10 classes
        self.fc2 = nn.Linear(nb_hidden, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2,stride=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2,stride=2))
        x = F.relu(self.fc1(x.view(-1, 2*2 * 64)))
        x = self.fc2(x)
        return x
    
class testNet2(nn.Module):
    def __init__(self,nb_hidden):
        super(testNet2, self).__init__()
        #takes 2x14x14
        #gives 32x12x12, 32x6x6 after maxpool
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3) 
        #takes 32x6x6, gives 64x6x6 then 64x3x3 after maxpool
        self.conv2 = nn.Conv2d(32,32, kernel_size=3)
        self.conv3 = nn.Conv2d(32,64,kernel_size=1)
        #takes 64*2*2 = 254, gives nb_hidden
        self.fc1 = nn.Linear(2*2*64, nb_hidden)
        #takes nb_hidden, gives 2 classes
        self.fc2 = nn.Linear(nb_hidden, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2,stride=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2,stride=2))
        x = F.elu(self.conv3(x),alpha = 0.7)
        x = F.relu(self.fc1(x.view(-1, 2*2 * 64)))
        x = self.fc2(x)
        return x
    
class testNet3(nn.Module):
    def __init__(self,nb_hidden):
        super(testNet3, self).__init__()
        #takes 2x14x14
        #gives 32x12x12, 32x6x6 after maxpool
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3) 
        #takes 32x6x6, gives 64x6x6 then 64x3x3 after maxpool
        self.conv2 = nn.Conv2d(32,32, kernel_size=3)
        self.conv3 = nn.Conv2d(32,64,kernel_size=1)
        #takes 64*2*2 = 254, gives nb_hidden
        self.fc1 = nn.Linear(2*2*64, nb_hidden)
        #takes nb_hidden, gives 2 classes
        self.fc2 = nn.Linear(nb_hidden, 2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(32)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2,stride=2))
        x = self.bn1(x)
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2,stride=2))
        x = self.bn2(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.fc1(x.view(-1, 2*2 * 64)))
        x = self.fc2(x)
        return x
    
class testNet4(nn.Module):
    def __init__(self,nb_hidden):
        super(testNet4, self).__init__()
        #takes 2x14x14
        #gives 32x12x12, 32x6x6 after maxpool
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3) 
        #takes 32x6x6, gives 64x6x6 then 64x3x3 after maxpool
        self.conv2 = nn.Conv2d(32,32, kernel_size=3)
        self.conv3 = nn.Conv2d(32,64,kernel_size=1)
        #takes 64*2*2 = 254, gives nb_hidden
        self.fc1 = nn.Linear(2*2*64, nb_hidden)
        #takes nb_hidden, gives 2 classes
        self.fc2 = nn.Linear(nb_hidden, 2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(32)
        self.dropout = nn.Dropout2d(0.38)
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2,stride=2))
        x = self.bn1(x)
        x = self.dropout(x)
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2,stride=2))
        x = self.bn2(x)
        x = self.dropout(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.fc1(x.view(-1, 2*2 * 64)))
        x = self.fc2(x)
        return x

### Train Model

In [8]:
def train_model(model, train_input, train_target, nb_epochs, eta, mini_batch_size):
    criterion = nn.CrossEntropyLoss()
    model.to(device)
    criterion.to(device)
    optimizer = optim.SGD(model.parameters(), lr = eta)
    losses = []
    
    for e in range(nb_epochs):
        for b in range(0, train_input.size(0), mini_batch_size):
            
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            losses.append(loss)
            model.zero_grad()
            loss.backward()
            optimizer.step()


def compute_nb_errors(model,data_input,data_target,mini_batch_size):
    nb_errors = 0;
    model.to(device)
    data_input, data_target = data_input.to(device),data_target.to(device)
    
    for b in range(0,data_input.size(0),mini_batch_size):
        output = model(data_input.narrow(0,b,mini_batch_size))
        _, predicted_classes = torch.max(output, 1)
        for k in range(mini_batch_size):
            if data_target[b+k]!=predicted_classes[k]:
                nb_errors += 1
    
    return nb_errors



In [9]:
#defining stuff and running
def run_test(model,nb_epochs,eta,mini_batch_size):
    print("""Using {} epochs, lr = {:.02f},Mini batch size = {}""".format(nb_epochs,eta,mini_batch_size))
    model.to(device)
    train_model(model, train_input, train_target,
                nb_epochs, eta, mini_batch_size)
    print('train_error {:.02f}% test_error {:.02f}% \n'.format(
                compute_nb_errors(model, train_input, train_target,mini_batch_size) / train_input.size(0) * 100,
                compute_nb_errors(model, test_input, test_target,mini_batch_size) / test_input.size(0) * 100
            )
        )

##### train_model with per epoch error print hidden here

In [63]:
#function to train AND print error per epoch
#just for test
def train_model_epoch_error(model, train_input, train_target, nb_epochs, eta, mini_batch_size):
    criterion = nn.CrossEntropyLoss()
    model.to(device)
    criterion.to(device)
    optimizer = optim.SGD(model.parameters(), lr = eta)
    losses = []
    
    for e in range(nb_epochs):
        nb_errors = 0;
        for b in range(0, train_input.size(0), mini_batch_size):
            
            output = model(train_input.narrow(0, b, mini_batch_size))
            _, predicted_classes = torch.max(output, 1)
            for k in range(mini_batch_size):
                if train_target[b+k]!=predicted_classes[k]:
                    nb_errors += 1
            
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            losses.append(loss)
            model.zero_grad()
            loss.backward()
            optimizer.step()
        print("Epoch : {}, train_error {:.02f}%".format(e,(100*(nb_errors/train_input.size(0)))))
            
#With epoch error
def run_test_epoch(model,nb_epochs,eta,mini_batch_size):
    print("Using {} epochs, lr = {:.02f}, Mini batch size = {}".format(nb_epochs,
                                                                        eta,
                                                                        mini_batch_size,
                                                                        ))
    model.to(device)
    train_model_epoch_error(model, train_input, train_target,
                nb_epochs, eta, mini_batch_size)
    
    print('train_error {:.02f}% test_error {:.02f}% \n'.format(
                compute_nb_errors(model, train_input, train_target,mini_batch_size) / train_input.size(0) * 100,
                compute_nb_errors(model, test_input, test_target,mini_batch_size) / test_input.size(0) * 100
            )
        )
   


In [64]:
nb_epochs, eta = 150, 1e-1

for mini_batch_size in [50,100]:
    for hidden in [10,25,50,100,200]:
        print("Hidden : {}".format(hidden))
        model = testNet(hidden)
        run_test_epoch(model,nb_epochs,eta,mini_batch_size)
        
print('done')

Hidden : 10
Using 150 epochs, lr = 0.10, Mini batch size = 50
Epoch : 0, train_error 45.10%
Epoch : 1, train_error 37.20%
Epoch : 2, train_error 30.60%
Epoch : 3, train_error 26.20%
Epoch : 4, train_error 22.80%
Epoch : 5, train_error 20.70%
Epoch : 6, train_error 18.00%
Epoch : 7, train_error 15.50%
Epoch : 8, train_error 17.60%
Epoch : 9, train_error 11.70%
Epoch : 10, train_error 14.70%
Epoch : 11, train_error 13.20%
Epoch : 12, train_error 9.60%
Epoch : 13, train_error 8.30%
Epoch : 14, train_error 7.50%
Epoch : 15, train_error 3.90%
Epoch : 16, train_error 6.60%
Epoch : 17, train_error 3.20%
Epoch : 18, train_error 0.50%
Epoch : 19, train_error 0.20%
Epoch : 20, train_error 0.10%
Epoch : 21, train_error 0.00%
Epoch : 22, train_error 0.00%
Epoch : 23, train_error 0.00%
Epoch : 24, train_error 0.00%
Epoch : 25, train_error 0.00%
Epoch : 26, train_error 0.00%
Epoch : 27, train_error 0.00%
Epoch : 28, train_error 0.00%
Epoch : 29, train_error 0.00%
Epoch : 30, train_error 0.00%
Epoch 

KeyboardInterrupt: 

In [10]:
nb_epochs, eta = 200, 1e-1

for mini_batch_size in [50,100]:
    for hidden in [10,25,50,100,200]:
        print("Hidden : {}".format(hidden))
        model = testNet2(hidden)
        run_test(model,nb_epochs,eta,mini_batch_size)
        
print('done')

Hidden : 10
Using 200 epochs, lr = 0.10,Mini batch size = 50
train_error 0.00% test_error 19.70% 

Hidden : 25
Using 200 epochs, lr = 0.10,Mini batch size = 50
train_error 0.00% test_error 20.80% 

Hidden : 50
Using 200 epochs, lr = 0.10,Mini batch size = 50
train_error 0.00% test_error 19.90% 

Hidden : 100
Using 200 epochs, lr = 0.10,Mini batch size = 50
train_error 0.00% test_error 19.40% 

Hidden : 200
Using 200 epochs, lr = 0.10,Mini batch size = 50
train_error 0.00% test_error 18.70% 

Hidden : 10
Using 200 epochs, lr = 0.10,Mini batch size = 100
train_error 0.00% test_error 18.00% 

Hidden : 25
Using 200 epochs, lr = 0.10,Mini batch size = 100
train_error 0.00% test_error 18.50% 

Hidden : 50
Using 200 epochs, lr = 0.10,Mini batch size = 100
train_error 0.00% test_error 19.20% 

Hidden : 100
Using 200 epochs, lr = 0.10,Mini batch size = 100
train_error 0.00% test_error 19.10% 

Hidden : 200
Using 200 epochs, lr = 0.10,Mini batch size = 100
train_error 0.00% test_error 17.30% 

d

In [14]:
nb_epochs, eta = 300, 1.1e-1

for mini_batch_size in [100]:
    for hidden in [4000]:
        print("Hidden : {}".format(hidden))
        model = testNet3(hidden)
        run_test(model,nb_epochs,eta,mini_batch_size)
        
print('done')

Hidden : 4000
Using 300 epochs, lr = 0.11,Mini batch size = 100
train_error 0.00% test_error 18.10% 

done


#### Test ResNet

In [18]:
class testResNetBlock(nn.Module):
    def __init__(self,nb_channels,kernel_size):
        super(testResNetBlock, self).__init__()

        self.conv1 = nn.Conv2d(nb_channels, nb_channels,
                               kernel_size = kernel_size,
                               padding = (kernel_size - 1) // 2)

        self.bn1 = nn.BatchNorm2d(nb_channels)

        self.conv2 = nn.Conv2d(nb_channels, nb_channels,
                               kernel_size = kernel_size,
                               padding = (kernel_size - 1) // 2)

        self.bn2 = nn.BatchNorm2d(nb_channels)


    def forward(self, x):
        y = self.conv1(x)
        y = self.bn1(y)
        y = F.relu(y)
        y = self.conv2(y)
        y = self.bn2(y)
        y = y + x
        y = F.relu(y)

        return y

######################################################################

class testResNet(nn.Module):

    def __init__(self, nb_residual_blocks, nb_channels,
                 kernel_size = 3):
        super(testResNet, self).__init__()

        self.conv0 = nn.Conv2d(2,nb_channels,kernel_size=3)
        self.resnet_blocks = nn.Sequential(
            *(testResNetBlock(nb_channels, kernel_size)
              for _ in range(nb_residual_blocks))
        )
        self.avg = nn.AvgPool2d(kernel_size=12)

        self.fc = nn.Linear(nb_channels, 2)

    def forward(self, x):
        x = F.relu(self.conv0(x))
        x = self.resnet_blocks(x)
        x = F.relu(self.avg(x))
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        return x
    
class testResNet2(nn.Module):

    def __init__(self, nb_residual_blocks, nb_channels,
                 kernel_size = 3):
        super(testResNet2, self).__init__()

        self.conv0 = nn.Conv2d(2,nb_channels,kernel_size=1)
        self.resnet_blocks = nn.Sequential(
            *(testResNetBlock(nb_channels, kernel_size)
              for _ in range(nb_residual_blocks))
        )
        self.avg = nn.AvgPool2d(kernel_size=14)

        self.fc = nn.Linear(nb_channels, 2)

    def forward(self, x):
        x = F.relu(self.conv0(x))
        nn.Dropout2d(0.15)
        x = self.resnet_blocks(x)
        x = F.relu(self.avg(x))
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        return x


In [16]:
nb_epochs, eta = 250, 0.9e-1

for mini_batch_size in [100]:
    for nb_blocks in [2,5,8]:
        for nb_channels in [20,40,60]:
            print("blocks {}, chans {}".format(nb_blocks,nb_channels))
            model = testResNet(nb_blocks,nb_channels)
            run_test(model,nb_epochs,eta,mini_batch_size)
        
print('done')

blocks 2, chans 20
Using 250 epochs, lr = 0.09,Mini batch size = 100
train_error 0.00% test_error 18.90% 

blocks 2, chans 40
Using 250 epochs, lr = 0.09,Mini batch size = 100
train_error 0.00% test_error 17.20% 

blocks 2, chans 60
Using 250 epochs, lr = 0.09,Mini batch size = 100
train_error 0.00% test_error 15.30% 

blocks 5, chans 20
Using 250 epochs, lr = 0.09,Mini batch size = 100
train_error 0.00% test_error 18.70% 

blocks 5, chans 40
Using 250 epochs, lr = 0.09,Mini batch size = 100
train_error 0.00% test_error 17.70% 

blocks 5, chans 60
Using 250 epochs, lr = 0.09,Mini batch size = 100
train_error 0.00% test_error 16.20% 

blocks 8, chans 20
Using 250 epochs, lr = 0.09,Mini batch size = 100
train_error 0.00% test_error 18.40% 

blocks 8, chans 40
Using 250 epochs, lr = 0.09,Mini batch size = 100
train_error 0.00% test_error 18.80% 

blocks 8, chans 60
Using 250 epochs, lr = 0.09,Mini batch size = 100
train_error 0.00% test_error 16.80% 

done


In [19]:
nb_epochs, eta = 250, 0.9e-1

for mini_batch_size in [100]:
    for nb_blocks in [2,5,8]:
        for nb_channels in [20,40,60]:
            print("blocks {}, chans {}".format(nb_blocks,nb_channels))
            model = testResNet2(nb_blocks,nb_channels)
            run_test(model,nb_epochs,eta,mini_batch_size)
        
print('done')

blocks 2, chans 20
Using 250 epochs, lr = 0.09,Mini batch size = 100
train_error 0.00% test_error 18.70% 

blocks 2, chans 40
Using 250 epochs, lr = 0.09,Mini batch size = 100
train_error 0.00% test_error 17.30% 

blocks 2, chans 60
Using 250 epochs, lr = 0.09,Mini batch size = 100
train_error 0.00% test_error 17.10% 

blocks 5, chans 20
Using 250 epochs, lr = 0.09,Mini batch size = 100
train_error 0.00% test_error 19.90% 

blocks 5, chans 40
Using 250 epochs, lr = 0.09,Mini batch size = 100
train_error 0.00% test_error 18.20% 

blocks 5, chans 60
Using 250 epochs, lr = 0.09,Mini batch size = 100
train_error 0.00% test_error 15.80% 

blocks 8, chans 20
Using 250 epochs, lr = 0.09,Mini batch size = 100
train_error 0.00% test_error 18.40% 

blocks 8, chans 40
Using 250 epochs, lr = 0.09,Mini batch size = 100
train_error 0.00% test_error 18.70% 

blocks 8, chans 60
Using 250 epochs, lr = 0.09,Mini batch size = 100
train_error 0.00% test_error 16.50% 

done


In [24]:
nb_epochs, eta = 200, 1e-1

for mini_batch_size in [50,100]:
    for hidden in [50,100,200,400]:
        print("Hidden : {}".format(hidden))
        model = testNet4(hidden)
        run_test(model,nb_epochs,eta,mini_batch_size)
        
print('done')

Hidden : 50
Using 200 epochs, lr = 0.10,Mini batch size = 50
train_error 1.30% test_error 18.50% 

Hidden : 100
Using 200 epochs, lr = 0.10,Mini batch size = 50
train_error 0.20% test_error 19.40% 

Hidden : 200
Using 200 epochs, lr = 0.10,Mini batch size = 50
train_error 0.20% test_error 17.70% 

Hidden : 400
Using 200 epochs, lr = 0.10,Mini batch size = 50
train_error 0.30% test_error 19.70% 

Hidden : 50
Using 200 epochs, lr = 0.10,Mini batch size = 100
train_error 0.20% test_error 19.00% 

Hidden : 100
Using 200 epochs, lr = 0.10,Mini batch size = 100
train_error 0.40% test_error 19.10% 

Hidden : 200
Using 200 epochs, lr = 0.10,Mini batch size = 100
train_error 1.00% test_error 19.50% 

Hidden : 400
Using 200 epochs, lr = 0.10,Mini batch size = 100
train_error 0.30% test_error 19.00% 

done
